In [2]:
import pandas as pd
import numpy as np

raw_df = pd.read_csv('train.csv')

raw_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
from sklearn import preprocessing
from sklearn.preprocessing import scale

def add_features(raw_df):
    raw_df['TotalSF'] = raw_df['1stFlrSF']+raw_df['2ndFlrSF']+raw_df['TotalBsmtSF']
    raw_df['OverallQCP'] = (raw_df['OverallQual']*raw_df['OverallCond'])/100
    raw_df['TotalBath'] = raw_df['FullBath']+raw_df['BsmtFullBath']

    #create two quant features for location (relevant to price)...median is better
    hood_names = raw_df['Neighborhood'].unique()
    hood_avg_prices = []
    hood_median_prices = []
    for name in hood_names:
        df_temp = raw_df[(raw_df['Neighborhood']==name)]
        avg_price = int(df_temp['SalePrice'].mean())
        hood_avg_prices.append(avg_price)
        median_price = df_temp['SalePrice'].median()
        hood_median_prices.append(median_price)
    #     print("{} has an average home sale price of {}".format(name,avg_price))

    name_avg_price_dict = dict(zip(hood_names, hood_avg_prices))
    raw_df['HoodAvg'] = raw_df['Neighborhood'].map(lambda x: name_avg_price_dict[x])
    name_med_price_dict = dict(zip(hood_names, hood_median_prices))
    raw_df['HoodMed'] = raw_df['Neighborhood'].map(lambda x: name_med_price_dict[x])

    #create quant feature for Functional
    func_dict ={
        'Typ':7,
        'Min1':6,
        'Min2':5,
        'Mod':4,
        'Maj1':3,
        'Maj2':2,
        'Sev':1,
        'Sal':0  
    }
    raw_df['FuncScore'] = raw_df['Functional'].map(lambda x: func_dict[x])
    return raw_df

In [4]:
cols = ['HoodAvg','HoodMed','TotalSF','OverallQCP','FuncScore','TotalBath','TotRmsAbvGrd','BedroomAbvGr','LotArea','GarageArea','SalePrice']

def standardize_foi(cols, foi_df):
    scaler = preprocessing.MinMaxScaler()
    for feature in cols:
        feature_mat = foi_df[feature].values.reshape(-1,1)
        foi_df.loc[:, feature] = scaler.fit_transform(feature_mat)
    return foi_df

In [5]:
def prepare_y(raw_df):
    raw_df.loc[raw_df['SaleCondition'] == 'Normal', 'SaleCondition'] = 0
    raw_df.loc[raw_df['SaleCondition'] != 0, 'SaleCondition'] = 1
    return raw_df['SaleCondition']

In [6]:
all_f_df = add_features(raw_df)
#print(all_f_df)

foi_df = all_f_df[list(cols)]
#print(foi_df.shape)
#print(foi_df)

s_foi_df = standardize_foi(cols, foi_df)

X = s_foi_df.copy()
y = prepare_y(raw_df)

lab_enc = preprocessing.LabelEncoder()
y_encoded = lab_enc.fit_transform(y)

# count = y.value_counts()
# print(count)

#X

/Users/ronp/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/Users/ronp/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/Users/ronp/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be s

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

model_lr = LogisticRegression()
model_lr.fit(X, y_encoded)
mean_acc = model_lr.score(X, y_encoded)

print("Logistical Regression - Imbalanced Mean Accuracy: %.3f" % mean_acc)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)
model_lr.fit(X_train, y_train)
y_pred = model_lr.predict(X_test)

print('Logistical Regression - Imbalanced Precision: %.3f' % precision_score(y_test, y_pred))
print('Logistical Regression - Imbalanced Recall: %.3f' % recall_score(y_test, y_pred))
print('Logistical Regression - Imbalanced Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Logistical Regression - Imbalanced F1 Score: %.3f' % f1_score(y_test, y_pred))

Logistical Regression - Imbalanced Mean Accuracy: 0.823
Logistical Regression - Imbalanced Precision: 1.000
Logistical Regression - Imbalanced Recall: 0.016
Logistical Regression - Imbalanced Accuracy: 0.788
Logistical Regression - Imbalanced F1 Score: 0.031


In [8]:
# import sys
# !{sys.executable} -m pip install imblearn

In [9]:
from imblearn import under_sampling, over_sampling
from imblearn.under_sampling import RandomUnderSampler

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

rus = RandomUnderSampler(random_state=0)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

model_lr_u = LogisticRegression()
model_lr_u.fit(X_train_rus, y_train_rus)
y_pred = model_lr_u.predict(X_test)

print('Logistical Regression - Undersampling Precision: %.3f' % precision_score(y_test, y_pred))
print('Logistical Regression - Undersampling Recall: %.3f' % recall_score(y_test, y_pred))
print('Logistical Regression - Undersampling Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Logistical Regression - Undersampling F1 Score: %.3f' % f1_score(y_test, y_pred))

Logistical Regression - Undersampling Precision: 0.296
Logistical Regression - Undersampling Recall: 0.587
Logistical Regression - Undersampling Accuracy: 0.610
Logistical Regression - Undersampling F1 Score: 0.394


In [10]:
from imblearn.over_sampling import RandomOverSampler

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

ros = RandomOverSampler(random_state=0)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

model_lr_o = LogisticRegression()
model_lr_o.fit(X_train_ros, y_train_ros)
y_pred = model_lr_o.predict(X_test)

print('Logistical Regression - Oversampling Precision: %.3f' % precision_score(y_test, y_pred))
print('Logistical Regression - Oversampling Recall: %.3f' % recall_score(y_test, y_pred))
print('Logistical Regression - Oversampling Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Logistical Regression - Oversampling F1 Score: %.3f' % f1_score(y_test, y_pred))

Logistical Regression - Oversampling Precision: 0.301
Logistical Regression - Oversampling Recall: 0.540
Logistical Regression - Oversampling Accuracy: 0.630
Logistical Regression - Oversampling F1 Score: 0.386


In [11]:
count = y.value_counts()
i_r = count[1]/count[0] # initial_ratio of minority to majority class
i_r

0.21869782971619364

In [12]:
best_accuracy_score = 0
best_f1_score = 0
best_min_samp_ratio = 0

for x in range(0,500):
    if ((1+x/100)*i_r) <= 1:
        X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

        rus_weighted = RandomUnderSampler(random_state=0, sampling_strategy=(1+x/100)*i_r)
        X_train_rus, y_train_rus = rus_weighted.fit_resample(X_train, y_train)

        model_lr_u = LogisticRegression()
        model_lr_u.fit(X_train_rus, y_train_rus)
        y_pred = model_lr_u.predict(X_test)

        current_accuracy_score = accuracy_score(y_test, y_pred)
        current_f1_score = f1_score(y_test, y_pred)
        current_min_samp_ratio = (1+x/100)*i_r

        if (current_accuracy_score*current_f1_score > best_accuracy_score*best_f1_score):
            best_accuracy_score = current_accuracy_score
            best_f1_score = current_f1_score
            best_min_samp_ratio = current_min_samp_ratio
        
print('When undersampling, the optimum minority class ratio: %.2f' % best_min_samp_ratio)

When undersampling, the optimum minority class ratio: 0.82


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

rus_weighted = RandomUnderSampler(random_state=0, sampling_strategy=0.82)
X_train_rus, y_train_rus = rus_weighted.fit_resample(X_train, y_train)

model_lr_u = LogisticRegression()
model_lr_u.fit(X_train_rus, y_train_rus)
y_pred = model_lr_u.predict(X_test)

print('Logistical Regression - Undersampling, weighted Precision: %.3f' % precision_score(y_test, y_pred))
print('Logistical Regression - Undersampling, weighted Recall: %.3f' % recall_score(y_test, y_pred))
print('Logistical Regression - Undersampling, weighted Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Logistical Regression - Undersampling, weighted F1 Score: %.3f' % f1_score(y_test, y_pred))

Logistical Regression - Undersampling, weighted Precision: 0.355
Logistical Regression - Undersampling, weighted Recall: 0.429
Logistical Regression - Undersampling, weighted Accuracy: 0.709
Logistical Regression - Undersampling, weighted F1 Score: 0.388


In [14]:
best_accuracy_score = 0
best_f1_score = 0
best_min_samp_ratio = 0

for x in range(0,500):
    if ((1+x/100)*i_r) <= 1:
        X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

        ros_weighted = RandomOverSampler(random_state=0, sampling_strategy=(1+x/100)*i_r)
        X_train_ros, y_train_ros = ros_weighted.fit_resample(X_train, y_train)

        model_lr_o = LogisticRegression()
        model_lr_o.fit(X_train_ros, y_train_ros)
        y_pred = model_lr_o.predict(X_test)

        current_accuracy_score = accuracy_score(y_test, y_pred)
        current_f1_score = f1_score(y_test, y_pred)
        current_min_samp_ratio = (1+x/100)*i_r

        if (current_accuracy_score*current_f1_score > best_accuracy_score*best_f1_score):
            best_accuracy_score = current_accuracy_score
            best_f1_score = current_f1_score
            best_min_samp_ratio = current_min_samp_ratio
        
print('When oversampling, the optimum minority class ratio: %.2f' % best_min_samp_ratio)

When oversampling, the optimum minority class ratio: 0.97


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

ros_weighted = RandomOverSampler(random_state=0, sampling_strategy=.97)
X_train_ros, y_train_ros = ros_weighted.fit_resample(X_train, y_train)

model_lr_o = LogisticRegression()
model_lr_o.fit(X_train_ros, y_train_ros)
y_pred = model_lr_o.predict(X_test)

print('Logistical Regression - Oversampling, weighted Precision: %.3f' % precision_score(y_test, y_pred))
print('Logistical Regression - Oversampling, weighted Recall: %.3f' % recall_score(y_test, y_pred))
print('Logistical Regression - Oversampling, weighted Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Logistical Regression - Oversampling, weighted F1 Score: %.3f' % f1_score(y_test, y_pred))

Logistical Regression - Oversampling, weighted Precision: 0.310
Logistical Regression - Oversampling, weighted Recall: 0.492
Logistical Regression - Oversampling, weighted Accuracy: 0.654
Logistical Regression - Oversampling, weighted F1 Score: 0.380


In [16]:
from sklearn.ensemble import RandomForestClassifier

best_accuracy_score = 0
best_f1_score = 0
best_n_estimator = 0

for x in range(1,100):

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

#     ros_weighted = RandomOverSampler(random_state=0, sampling_strategy=(1+x/100)*i_r)
#     X_train_ros, y_train_ros = ros_weighted.fit_resample(X_train, y_train)

    rfc = RandomForestClassifier(n_estimators=x, random_state=0)
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict(X_test)
    
    current_accuracy_score = accuracy_score(y_test, y_pred)
    current_f1_score = f1_score(y_test, y_pred)

    if (current_accuracy_score*current_f1_score > best_accuracy_score*best_f1_score):
        best_accuracy_score = current_accuracy_score
        best_f1_score = current_f1_score
        best_n_estimator = x
        
print('When imbalanced, the optimum number of estimators is', best_n_estimator)

When imbalanced, the optimum number of estimators is 13


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

rfc = RandomForestClassifier(n_estimators=13, random_state=0)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

print('Random Forest Classifier - Imbalanced Precision: %.3f' % precision_score(y_test, y_pred))
print('Random Forest Classifier - Imbalanced Recall: %.3f' % recall_score(y_test, y_pred))
print('Random Forest Classifier - Imbalanced Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Random Forest Classifier - Imbalanced F1 Score: %.3f' % f1_score(y_test, y_pred))

Random Forest Classifier - Imbalanced Precision: 0.565
Random Forest Classifier - Imbalanced Recall: 0.206
Random Forest Classifier - Imbalanced Accuracy: 0.795
Random Forest Classifier - Imbalanced F1 Score: 0.302


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

rus = RandomUnderSampler(random_state=0)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

rfc_u = RandomForestClassifier(n_estimators=13, random_state=0)
rfc_u.fit(X_train_rus, y_train_rus)
y_pred = rfc_u.predict(X_test)

print('Random Forest Classifier - Undersampling Precision: %.3f' % precision_score(y_test, y_pred))
print('Random Forest Classifier - Undersampling Recall: %.3f' % recall_score(y_test, y_pred))
print('Random Forest Classifier - Undersampling Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Random Forest Classifier - Undersampling F1 Score: %.3f' % f1_score(y_test, y_pred))

Random Forest Classifier - Undersampling Precision: 0.286
Random Forest Classifier - Undersampling Recall: 0.571
Random Forest Classifier - Undersampling Accuracy: 0.599
Random Forest Classifier - Undersampling F1 Score: 0.381


In [33]:
best_accuracy_score = 0
best_f1_score = 0
best_min_samp_ratio = 0
best_n_estimator = 1

for y in range(1,25):
    for x in range(100,550):
        if ((x/100)*i_r) <= 1:
            X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

            rus_weighted = RandomUnderSampler(random_state=0, sampling_strategy=(x/100)*i_r)
            X_train_rus, y_train_rus = rus_weighted.fit_resample(X_train, y_train)

            rfc_u = RandomForestClassifier(n_estimators=y, random_state=0)
            rfc_u.fit(X_train_rus, y_train_rus)
            y_pred = rfc_u.predict(X_test)

            current_accuracy_score = accuracy_score(y_test, y_pred)
            current_f1_score = f1_score(y_test, y_pred)
            current_min_samp_ratio = (x/100)*i_r

            if (current_accuracy_score*current_f1_score > best_accuracy_score*best_f1_score):
                best_accuracy_score = current_accuracy_score
                best_f1_score = current_f1_score
                best_min_samp_ratio = current_min_samp_ratio
                best_n_estimator = y
        
print('When undersampling, the optimum minority class ratio: %.2f' % best_min_samp_ratio)
print('When undersampling, the optimum n_estimator:', best_n_estimator)

When undersampling, the optimum minority class ratio: 0.52
When undersampling, the optimum n_estimator: 21


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

rus = RandomUnderSampler(random_state=0, sampling_strategy=0.52)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

rfc_u = RandomForestClassifier(n_estimators=21, random_state=0)
rfc_u.fit(X_train_rus, y_train_rus)
y_pred = rfc_u.predict(X_test)

print('Random Forest Classifier - Undersampling, weighted Precision: %.3f' % precision_score(y_test, y_pred))
print('Random Forest Classifier - Undersampling, weighted Recall: %.3f' % recall_score(y_test, y_pred))
print('Random Forest Classifier - Undersampling, weighted Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Random Forest Classifier - Undersampling, weighted F1 Score: %.3f' % f1_score(y_test, y_pred))

Random Forest Classifier - Undersampling, weighted Precision: 0.446
Random Forest Classifier - Undersampling, weighted Recall: 0.397
Random Forest Classifier - Undersampling, weighted Accuracy: 0.764
Random Forest Classifier - Undersampling, weighted F1 Score: 0.420


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

ros = RandomOverSampler(random_state=0)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

rfc_o = RandomForestClassifier(n_estimators=13, random_state=0)
rfc_o.fit(X_train_ros, y_train_ros)
y_pred = rfc_o.predict(X_test)

print('Random Forest Classifier - Oversampling Precision: %.3f' % precision_score(y_test, y_pred))
print('Random Forest Classifier - Oversampling Recall: %.3f' % recall_score(y_test, y_pred))
print('Random Forest Classifier - Oversampling Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Random Forest Classifier - Oversampling F1 Score: %.3f' % f1_score(y_test, y_pred))

Random Forest Classifier - Oversampling Precision: 0.341
Random Forest Classifier - Oversampling Recall: 0.222
Random Forest Classifier - Oversampling Accuracy: 0.740
Random Forest Classifier - Oversampling F1 Score: 0.269


In [22]:
best_accuracy_score = 0
best_f1_score = 0
best_min_samp_ratio = 0
best_n_estimator = 1

for y in range(1,20):
    for x in range(100,550):
        if ((x/100)*i_r) <= 1:
            X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

            ros_weighted = RandomOverSampler(random_state=0, sampling_strategy=(x/100)*i_r)
            X_train_ros, y_train_ros = ros_weighted.fit_resample(X_train, y_train)

            rfc_o = RandomForestClassifier(n_estimators=y, random_state=0)
            rfc_o.fit(X_train_ros, y_train_ros)
            y_pred = rfc_o.predict(X_test)

            current_accuracy_score = accuracy_score(y_test, y_pred)
            current_f1_score = f1_score(y_test, y_pred)
            current_min_samp_ratio = (x/100)*i_r

            if (current_accuracy_score*current_f1_score > best_accuracy_score*best_f1_score):
                best_accuracy_score = current_accuracy_score
                best_f1_score = current_f1_score
                best_min_samp_ratio = current_min_samp_ratio
                best_n_estimator = y
        
print('When oversampling, the optimum minority class ratio: %.2f' % best_min_samp_ratio)
print('When oversampling, the optimum n_estimator:', best_n_estimator)

When oversampling, the optimum minority class ratio: 0.97
When oversampling, the optimum n_estimator: 7


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

ros_weighted = RandomOverSampler(random_state=0, sampling_strategy=0.97)
X_train_ros, y_train_ros = ros_weighted.fit_resample(X_train, y_train)

rfc_o = RandomForestClassifier(n_estimators=7, random_state=0)
rfc_o.fit(X_train_ros, y_train_ros)
y_pred = rfc_o.predict(X_test)

print('Random Forest Classifier - Oversampling, weighted Precision: %.3f' % precision_score(y_test, y_pred))
print('Random Forest Classifier - Oversampling, weighted Recall: %.3f' % recall_score(y_test, y_pred))
print('Random Forest Classifier - Oversampling, weighted Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Random Forest Classifier - Oversampling, weighted F1 Score: %.3f' % f1_score(y_test, y_pred))

Random Forest Classifier - Oversampling, weighted Precision: 0.545
Random Forest Classifier - Oversampling, weighted Recall: 0.381
Random Forest Classifier - Oversampling, weighted Accuracy: 0.798
Random Forest Classifier - Oversampling, weighted F1 Score: 0.449


In [35]:
from sklearn.neural_network import MLPClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

nn = MLPClassifier(max_iter=1000, activation="relu", hidden_layer_sizes=(10,10))
nn.fit(X_train, y_train)
y_pred = nn.predict(X_test)

print('Nueral Network (MLPClassifier) - Imbalanced Precision: %.3f' % precision_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Imbalanced Recall: %.3f' % recall_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Imbalanced Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Imbalanced F1 Score: %.3f' % f1_score(y_test, y_pred))

Nueral Network (MLPClassifier) - Imbalanced Precision: 0.600
Nueral Network (MLPClassifier) - Imbalanced Recall: 0.048
Nueral Network (MLPClassifier) - Imbalanced Accuracy: 0.788
Nueral Network (MLPClassifier) - Imbalanced F1 Score: 0.088


In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

rus = RandomUnderSampler(random_state=0)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

nn = MLPClassifier(max_iter=1000, activation="relu", alpha=0.01, hidden_layer_sizes=(10,10))
nn.fit(X_train_rus, y_train_rus)
y_pred = nn.predict(X_test)

print('Nueral Network (MLPClassifier) - Undersampling Precision: %.3f' % precision_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Undersampling Recall: %.3f' % recall_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Undersampling Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Undersampling F1 Score: %.3f' % f1_score(y_test, y_pred))

Nueral Network (MLPClassifier) - Undersampling Precision: 0.296
Nueral Network (MLPClassifier) - Undersampling Recall: 0.508
Nueral Network (MLPClassifier) - Undersampling Accuracy: 0.634
Nueral Network (MLPClassifier) - Undersampling F1 Score: 0.374


In [49]:
best_accuracy_score = 0
best_f1_score = 0
best_min_samp_ratio = 0
best_hidden_layer_sizes = 1

for y in range(1,20):
    for x in range(20,55):
        if ((x/10)*i_r) <= 1:
            X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

            rus_weighted = RandomUnderSampler(random_state=0, sampling_strategy=(x/10)*i_r)
            X_train_rus, y_train_rus = rus_weighted.fit_resample(X_train, y_train)

            nn_u = MLPClassifier(max_iter=1100, alpha=0.01, activation="relu", hidden_layer_sizes=(y, y))
            nn_u.fit(X_train_rus, y_train_rus)
            y_pred = nn_u.predict(X_test)

            current_accuracy_score = accuracy_score(y_test, y_pred)
            current_f1_score = f1_score(y_test, y_pred)
            current_min_samp_ratio = (x/10)*i_r

            if (current_accuracy_score*current_f1_score > best_accuracy_score*best_f1_score):
                best_accuracy_score = current_accuracy_score
                best_f1_score = current_f1_score
                best_min_samp_ratio = current_min_samp_ratio
                best_hidden_layer_sizes = y
        
print('When undersampling, the optimum minority class ratio: %.2f' % best_min_samp_ratio)
print('When undersampling, the optimum hidden layer sizes:', best_hidden_layer_sizes)

/Users/ronp/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ronp/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ronp/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ronp/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1100) reached and the optimization hasn't converged yet.
  warnings.warn(


When undersampling, the optimum minority class ratio: 0.59
When undersampling, the optimum hidden layer sizes: 19


In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

rus = RandomUnderSampler(random_state=0, sampling_strategy=0.59)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

nn = MLPClassifier(max_iter=1000, activation="relu", alpha=0.01, hidden_layer_sizes=(19,19))
nn.fit(X_train_rus, y_train_rus)
y_pred = nn.predict(X_test)

print('Nueral Network (MLPClassifier) - Undersampling, weighted Precision: %.3f' % precision_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Undersampling, weighted Recall: %.3f' % recall_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Undersampling, weighted Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Undersampling, weighted F1 Score: %.3f' % f1_score(y_test, y_pred))

Nueral Network (MLPClassifier) - Undersampling, weighted Precision: 0.386
Nueral Network (MLPClassifier) - Undersampling, weighted Recall: 0.349
Nueral Network (MLPClassifier) - Undersampling, weighted Accuracy: 0.740
Nueral Network (MLPClassifier) - Undersampling, weighted F1 Score: 0.367


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

ros = RandomOverSampler(random_state=0)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

nn = MLPClassifier(max_iter=1000, activation="relu", hidden_layer_sizes=(10,10))
nn.fit(X_train_ros, y_train_ros)
y_pred = nn.predict(X_test)

print('Nueral Network (MLPClassifier) - Oversampling Precision: %.3f' % precision_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Oversampling Recall: %.3f' % recall_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Oversampling Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Oversampling F1 Score: %.3f' % f1_score(y_test, y_pred))

Nueral Network (MLPClassifier) - Oversampling Precision: 0.337
Nueral Network (MLPClassifier) - Oversampling Recall: 0.508
Nueral Network (MLPClassifier) - Oversampling Accuracy: 0.678
Nueral Network (MLPClassifier) - Oversampling F1 Score: 0.405


In [50]:
best_accuracy_score = 0
best_f1_score = 0
best_min_samp_ratio = 0
best_hidden_layer_sizes = 1

for y in range(1,20):
    for x in range(20,55):
        if ((x/10)*i_r) <= 1:
            X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

            ros_weighted = RandomOverSampler(random_state=0, sampling_strategy=(x/10)*i_r)
            X_train_ros, y_train_ros = ros_weighted.fit_resample(X_train, y_train)

            nn_o = MLPClassifier(max_iter=1100, alpha=0.01, activation="relu", hidden_layer_sizes=(y, y))
            nn_o.fit(X_train_ros, y_train_ros)
            y_pred = nn_o.predict(X_test)

            current_accuracy_score = accuracy_score(y_test, y_pred)
            current_f1_score = f1_score(y_test, y_pred)
            current_min_samp_ratio = (x/10)*i_r

            if (current_accuracy_score*current_f1_score > best_accuracy_score*best_f1_score):
                best_accuracy_score = current_accuracy_score
                best_f1_score = current_f1_score
                best_min_samp_ratio = current_min_samp_ratio
                best_hidden_layer_sizes = y
        
print('When oversampling, the optimum minority class ratio: %.2f' % best_min_samp_ratio)
print('When oversampling, the optimum hidden layer sizes:', best_hidden_layer_sizes)

When undersampling, the optimum minority class ratio: 0.72
When undersampling, the optimum hidden layer sizes: 17


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=0)

ros_weighted = RandomOverSampler(random_state=0, sampling_strategy=0.72)
X_train_ros, y_train_ros = ros_weighted.fit_resample(X_train, y_train)

nn = MLPClassifier(max_iter=1100, activation="relu", alpha=0.01, hidden_layer_sizes=(17,17))
nn.fit(X_train_ros, y_train_ros)
y_pred = nn.predict(X_test)

print('Nueral Network (MLPClassifier) - Oversampling, weighted Precision: %.3f' % precision_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Oversampling, weighted Recall: %.3f' % recall_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Oversampling, weighted Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Nueral Network (MLPClassifier) - Oversampling, weighted F1 Score: %.3f' % f1_score(y_test, y_pred))

Nueral Network (MLPClassifier) - Oversampling, weighted Precision: 0.358
Nueral Network (MLPClassifier) - Oversampling, weighted Recall: 0.302
Nueral Network (MLPClassifier) - Oversampling, weighted Accuracy: 0.733
Nueral Network (MLPClassifier) - Oversampling, weighted F1 Score: 0.328
